In [ ]:
# Importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import QuantileTransformer
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from scipy.cluster import hierarchy
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit

%matplotlib inline

In [ ]:
df = pd.read_csv('imagesPointData.csv')

In [ ]:
df.head()

In [ ]:
X = df.iloc[:,0:-7]
X

In [ ]:
y = df.iloc[:,-7:-4]
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [ ]:
r1 = MultiOutputRegressor(GradientBoostingRegressor(random_state=0)),
r2 = MultiOutputRegressor(RandomForestRegressor(random_state=0)),
r3 = MultiOutputRegressor(LinearRegression()),
voting_clf = make_pipeline(StandardScaler(),
        MultiOutputRegressor(VotingRegressor(
            estimators=[('gb', GradientBoostingRegressor(random_state=0,n_estimators=500,learning_rate=0.1,max_depth=6)),
             ('rf', RandomForestRegressor(n_estimators=250)),
             ]))
)
voting_clf.fit(X_train.values, y_train.values)
predictions =  voting_clf.predict(X_train.values)




print(mean_squared_error(y_train.values, predictions))
print(r2_score(y_train.values, predictions))
test_predictions =  voting_clf.predict(X_test.values)
print(mean_squared_error(y_test, test_predictions))
print(r2_score(y_test, test_predictions))

In [ ]:
pit = test_predictions[0,0]
ya = test_predictions[0,1]
ro = test_predictions[0,2]
print(pit, ya, ro)



def draw_axis(img, yaw, pitch, roll, tdx=None, tdy=None, size = 100):

    pitch = pitch * np.pi / 180
    yaw = -(yaw * np.pi / 180)
    roll = roll * np.pi / 180

    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img



predictor=dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
cnn_face_detector = dlib.get_frontal_face_detector()


img = sio.loadmat('image00010.mat')


image='image00010.jpg'
frame = cv2.imread(image)

cv2_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

dets = cnn_face_detector(cv2_frame, 1)
for (idx, det) in enumerate(dets):
            # Get x_min, y_min, x_max, y_max, conf
            x_min = det.left()
            y_min = det.top()
            x_max = det.right()-x_min
            y_max = det.bottom()-y_min
            # conf = det.confidence
            # print(det)

            shape = predictor(cv2_frame, det)
            tdx,tdy=shape.part(30).x, shape.part(30).y
            bbox_height, bbox_width = frame.shape[:2]
           
            cv2.destroyAllWindows()
            cv2_imshow(draw_axis(frame,pit,yaw ,rol,tdx,tdy)
            )
cv2.destroyAllWindows()

